## Bibliotecas

In [78]:
import joblib
import pandas as pd
import os

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVR

## Importación datos

In [79]:
CSV_FILE = 'EDA/df_normalized.csv'

df = pd.read_csv(CSV_FILE)
display(df.head())

,es_numerosa,educacion_madre,educacion_padre,tiempo_viaje,tiempo_estudio,consumo_alcohol_entre_semana,consumo_alcohol_fin_de_semana,suspensos,faltas,nota1,nota2,nota3,educacion_media,notas_media
0,0.0,1.00,0.75,0.0,1.0,0.0,0.0,0.0,0.0,0.615385,0.6,0.583333,0.875,0.62500
1,1.0,0.50,0.50,0.5,1.0,0.0,0.0,0.0,0.5,0.615385,0.6,0.666667,0.500,0.65625
2,1.0,1.00,0.50,0.5,0.5,0.0,0.0,0.0,0.9,0.384615,0.2,0.250000,0.750,0.28125
3,1.0,0.50,0.25,0.5,0.5,0.0,0.2,0.0,0.2,0.461538,0.4,0.666667,0.375,0.53125
4,0.0,0.25,0.50,0.5,0.0,0.0,0.6,0.0,0.0,0.615385,0.6,0.666667,0.375,0.65625


## Eliminación de variables inecesarias

In [80]:
#n² 94%

df = df.drop(
    columns = [
        'educacion_padre', 'educacion_madre',
        'nota1', 'nota2', 
    ]
)

# df

## Modelo notas_media
Este modelo se centra en predecir la `nota_media` de un estudiante

In [81]:
# Dividir los datos en conjuntos de entrenamiento y prueba
train_df, test_df = train_test_split(
    df,
    test_size=0.3
)

X_train = train_df.drop(columns=['notas_media'])
Y_train = train_df["notas_media"]
X_test = test_df.drop(columns=['notas_media'])
Y_test = test_df["notas_media"]

# display(X_train)
# display(Y_train)
# display(X_test)
# display(X_test)

## Modelos

### Regresión Linear

La regresión lineal es una técnica más que conocida de modelado predictivo que se utiliza para predecir el valor de una variable dependiente (`Y`)  basada en una o más variables independientes (`X`).

#### Variables que influyen en el modelo

- **Variable Dependiente (Y)**: Es la variable que queremos predecir.
- **Variable Independiente (X)**: Son las variables que utilizamos para hacer la predicción.
- **Coeficientes (β)**: Representan la relación entre las variables independientes y la variable dependiente.
- **Intersección (β0)**: Es el valor de Y cuando todas las variables independientes son cero.

#### Fórmula de la Regresión Lineal

La ecuación de la regresión lineal simple (con una sola variable independiente) es:

\[ Y = β0 + β1X \]

Para la regresión lineal múltiple (con múltiples variables independientes), la ecuación es:

\[ Y = β0 + β1X1 + β2X2 + ... + βnXn \]

### Ventajas y Desventajas

#### Ventajas
- **Simplicidad**: Fácil de entender e interpretar.
- **Eficiencia**: Rápido de entrenar y predecir.
- **Interpretabilidad**: Los coeficientes pueden interpretarse directamente.

#### Desventajas
- **Linealidad**: Asume una relación lineal entre las variables independientes y dependientes.
- **Sensibilidad a Outliers**: Los valores atípicos pueden influir significativamente en el modelo.
- **Multicolinealidad**: La presencia de alta correlación entre las variables independientes puede afectar la estabilidad del modelo.

In [82]:
# Definir modelo
lr_reg = LinearRegression()

# Definimos los hiperámetros dentro del diccionario param_grid
# Este será udano por 
param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'positive': [True, False]
}

# La instacia de GridSearchCV sirve para hacer una búsqueda 
# Estensiva sobre los diferentes hiperparámtros especificados en param_grid
grid_search = GridSearchCV(
    # Modelo elegido
    lr_reg,
    # Hiperparaḿetros
    param_grid=param_grid,
    # Número de cruces para la validación cruzada
    cv=5,
    # Métrica evaluación modelo
    scoring='r2'
)

### Entrenamiento

In [83]:
# Entrenamiento
grid_search.fit(X_train, Y_train)

# Escogemos las méjores métricas
best_lr_reg = grid_search.best_estimator_

# Predecir
Y_pred = best_lr_reg.predict(X_test)

# Otras métricas útiles
mae_lr = mean_absolute_error(Y_test, Y_pred)
mse_lr = mean_squared_error(Y_test, Y_pred)

### Resultados modelo

In [84]:
# Mostrar las métricas de validación cruzada
print(f'Mejores parámteos: {grid_search.best_params_}')
print(f'Mejor resultado validación cruzada (R²): {grid_search.best_score_:.4f}')

# Otras métricas
print(f'MAE (Error absoluto medio): {mae_lr:.4f}')
print(f'MSE (Error cuadrático medio): {mse_lr:.4f}')

Mejores parámteos: {'copy_X': True, 'fit_intercept': False, 'positive': False}
Mejor resultado validación cruzada (R²): 0.9387
MAE (Error absoluto medio): 0.0475
MSE (Error cuadrático medio): 0.0036


### Gradient Boosting

**Gradient Boosting** es un método de aprendizaje automático que combina múltiples modelos simples (o débiles) para crear un modelo predictivo fuerte.

#### Datos de Entrada

1. **Características (X):** Variables predictoras del modelo. Pueden ser numéricas o categóricas.
2. **Etiqueta (Y):** Variable objetivo que se desea predecir. Puede ser continua (para regresión) o categórica (para clasificación).

#### Fórmula Básica

El modelo predictivo final se construye iterativamente sumando modelos simples ajustados al error residual. La fórmula general es:

\[
F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x)
\]

Donde:
- \( F_m(x) \): Modelo en la iteración \( m \).
- \( F_{m-1}(x) \): Modelo acumulado hasta la iteración \( m-1 \).
- \( \eta \): Tasa de aprendizaje que controla la contribución de cada modelo.
- \( h_m(x) \): Modelo ajustado al error residual en la iteración \( m \).

#### Ventajas

- **Precisión:** Generalmente proporciona alta precisión en comparación con otros métodos.
- **Flexibilidad:** Puede manejar diferentes tipos de datos y problemas.
- **No requiere normalización:** Los datos no necesitan ser normalizados.

#### Desventajas

- **Tiempo de Entrenamiento:** Puede ser lento debido a la naturaleza iterativa del algoritmo.
- **Complejidad:** Puede ser más complejo de entender y ajustar en comparación con otros métodos.
- **Sobreajuste:** Puede sobreajustarse si no se controla adecuadamente.

#### Parámetros Clave

- `n_estimators`: Número de árboles en el modelo.
- `learning_rate`: Tasa de aprendizaje que controla el peso de cada árbol.
- `max_depth`: Profundidad máxima de cada árbol.

##### Referencia
Scikit-learn. (n.d.). GradientBoostingClassifier. Scikit-learn Documentation. Retrieved December 6, 2024, from https://scikit-learn.org/1.5/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html


### Configuración hiperparámetros

In [85]:
# Definir el espacio de parámetros
param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Configurar la búsqueda en cuadrícula
grid_search_gb = GridSearchCV(
    GradientBoostingRegressor(),
    param_grid=param_grid_gb,
    cv=5,
    scoring='r2',
)

### Entrenamiento

In [86]:
# Realizar la búsqueda en cuadrícula
grid_search_gb.fit(X_train, Y_train)

# Obtener las mejores estimaciones
best_gb_reg = grid_search_gb.best_estimator_

# Predecir
Y_pred_best_gb = best_gb_reg.predict(X_test)

# Otras métricas interesantas
mae_best_gb = mean_absolute_error(Y_test, Y_pred_best_gb)
mse_best_gb = mean_squared_error(Y_test, Y_pred_best_gb)

### Resultados modelo

In [87]:
# Mostrar las métricas
print(f'Mejores hiperparámetros encontrados: {grid_search_gb.best_params_}')
print(f'Mejor resultado con validación cruzada (métrica R²): {grid_search_gb.best_score_:.4f}')
print(f'MAE (Error absoluto medio): {mae_best_gb:.4f}')
print(f'MSE (Error cuadrático medio): {mse_best_gb:.4f}')

Mejores hiperparámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
Mejor resultado con validación cruzada (métrica R²): 0.9318
MAE (Error absoluto medio): 0.0488
MSE (Error cuadrático medio): 0.0038


### Ridge

**Ridge Regression** es una técnica de aprendizaje supervisado que extiende la regresión lineal al añadir un término de penalización. Este término ayuda a reducir el sobreajuste y mejora la capacidad de generalización del modelo al restringir los coeficientes de las características.

#### Funcionamiento

Ridge Regression minimiza una función de pérdida que combina el error cuadrático medio y una penalización L2 sobre los coeficientes. La fórmula general es:


\[
\text{Error} = \sum_{i=1}^{n}(y_i - \hat{y}_i)^2 + \alpha \cdot \sum_{j=1}^{p}\beta_j^2
\]

Donde:
- \( y_i \): Valores reales de la variable objetivo.
- \( \hat{y}_i \): Predicciones del modelo.
- \( \beta_j \): Coeficientes de las características.
- \( \alpha \): Parámetro de regularización que controla la penalización (cuanto mayor sea \( \alpha \), mayor será la regularización).

#### Parámetros Clave

1. **`alpha`**: Controla la intensidad de la regularización.
   - Un valor mayor de `alpha` aplica una penalización más fuerte.
   - Un valor menor de `alpha` se acerca a una regresión lineal estándar.
   
2. **`fit_intercept`**: Determina si se ajusta un término independiente (intercepto) en el modelo.
   
3. **`normalize`**: Si es `True`, las características se normalizan antes de ajustar el modelo.

### Hiperparámetros

In [88]:
param_grid_ridge = {
    'alpha': [0.1, 1.0, 10.0],
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'positive': [True, False]
}

grid_search_ridge = GridSearchCV(
    Ridge(),
    param_grid=param_grid_ridge,
    cv=5,
    scoring='r2',
)

### Entrenamiento

In [89]:
grid_search_ridge.fit(X_train, Y_train)

best_ridge_grid = grid_search_ridge.best_estimator_

# Predecir
Y_pred_ridge_grid = best_ridge_grid.predict(X_test)

# Evaluación
mae_ridge_grid = mean_absolute_error(Y_test, Y_pred_ridge_grid)
mse_ridge_grid = mean_squared_error(Y_test, Y_pred_ridge_grid)

### Evaluación modelo

In [90]:
# Mostrar las métricas de validación cruzada
print(f'Mejores hipeparámetros: {grid_search_ridge.best_params_}')
print(f'Mejor resultado con validación cruzada (métrica R²): {grid_search_ridge.best_score_:.4f}')
print(f'MAE (Error absoluto medio): {mae_ridge_grid:.4f}')
print(f'MSE (Error cuadrático medio): {mse_ridge_grid:.4f}')

Mejores hipeparámetros: {'alpha': 0.1, 'copy_X': True, 'fit_intercept': False, 'positive': False}
Mejor resultado con validación cruzada (métrica R²): 0.9384
MAE (Error absoluto medio): 0.0476
MSE (Error cuadrático medio): 0.0037


### Support Vector Regression (SVR)

**Support Vector Regression (SVR)** es una técnica de aprendizaje supervisado basada en máquinas de soporte vectorial (**SVM**) que se utiliza para problemas de regresión. Su objetivo es encontrar una función que tenga, al mismo tiempo, la mayor simplicidad y el menor error dentro de un margen tolerable definido por el usuario.

#### ¿Cómo funciona?

1. **Margen de tolerancia (\(\epsilon\))**:
   - SVR permite que los errores de predicción dentro de un margen definido (\(\epsilon\)) no sean penalizados.
   - Cualquier error fuera de este margen sí se penaliza.

2. **Optimización**:
   - Busca minimizar una función de pérdida basada en el margen de tolerancia y un término de regularización que controla la complejidad del modelo.
   - La solución se define en términos de un subconjunto de los puntos de datos llamados **vectores de soporte**.

#### Fórmula Básica

El objetivo de SVR es resolver el siguiente problema de optimización:

\[
\min_{w, b} \frac{1}{2} \|w\|^2 + C \sum_{i=1}^n (\xi_i + \xi_i^*)
\]

Sujeto a las restricciones:
\[
y_i - (w^T x_i + b) \leq \epsilon + \xi_i
\]
\[
(w^T x_i + b) - y_i \leq \epsilon + \xi_i^*
\]
\[
\xi_i, \xi_i^* \geq 0
\]

Donde:
- \( w \): Vector de pesos.
- \( b \): Término independiente.
- \( C \): Parámetro de regularización que controla el equilibrio entre el error tolerado y la simplicidad del modelo.
- \( \xi_i, \xi_i^* \): Variables de holgura que permiten errores fuera del margen.
- \( \epsilon \): Margen de tolerancia.

#### Ventajas

- **Control de Complejidad**: Gracias al parámetro \(C\), se puede ajustar el equilibrio entre simplicidad del modelo y error.
- **No Linealidad**: A través del uso de kernels, SVR puede capturar relaciones no lineales en los datos.
- **Robustez**: Es menos propenso al sobreajuste en comparación con otros métodos de regresión, especialmente en conjuntos de datos pequeños.

#### Desventajas

- **Sensibilidad a los Parámetros**: La selección de \(C\), \(\epsilon\) y el tipo de kernel afecta significativamente el rendimiento.
- **Coste Computacional**: Para grandes conjuntos de datos, puede ser más lento en comparación con otros métodos.
- **Interpretabilidad**: Los modelos basados en kernels pueden ser difíciles de interpretar.


#### Parámetros Clave

1. **`kernel`**: Define la función de kernel utilizada para proyectar los datos a un espacio de características más alto. Los más comunes son:
   - `linear`
   - `poly`
   - `rbf` (Radial Basis Function)
   - `sigmoid`

2. **`C`**: Parámetro de regularización que controla el trade-off entre simplicidad del modelo y tolerancia al error.

3. **`epsilon`**: Define el margen de tolerancia donde los errores no se penalizan.

4. **`gamma`**: Utilizado en kernels no lineales como `rbf` y `poly` para controlar la influencia de un solo punto de datos.


In [91]:
# Definir el espacio de parámetros
param_grid_svr = {
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 0.2],
    'kernel': ['linear', 'poly', 'rbf']
}

# Configurar la búsqueda en cuadrícula
grid_search_svr = GridSearchCV(
    SVR(),
    param_grid=param_grid_svr,
    cv=5,
    scoring='r2',
)

In [92]:
grid_search_svr.fit(X_train, Y_train)
best_svr = grid_search_svr.best_estimator_

# Predecir
Y_pred_svr = best_svr.predict(X_test)

In [93]:
# Evaluación
mae_svr = mean_absolute_error(Y_test, Y_pred_svr)
mse_svr = mean_squared_error(Y_test, Y_pred_svr)

# Mostrar las métricas de validación cruzada
print(f'Mejores hiperparámetros encontrados: {grid_search_svr.best_params_}')
print(f'Mejor resultado con validación cruzada (métrica R²): {grid_search_svr.best_score_:.4f}')
print(f'MAE (Error absoluto medio): {mae_svr:.4f}')
print(f'MSE (Error cuadrático medio): {mse_svr:.4f}')

Mejores hiperparámetros encontrados: {'C': 1, 'epsilon': 0.01, 'kernel': 'linear'}
Mejor resultado con validación cruzada (métrica R²): 0.9372
MAE (Error absoluto medio): 0.0476
MSE (Error cuadrático medio): 0.0036


## Resultados finales

In [94]:
# Calcular el R² para cada modelo
r2_lr = grid_search.score(X_test, Y_test)
r2_gb = grid_search_gb.score(X_test, Y_test)
r2_ridge = grid_search_ridge.score(X_test, Y_test)
r2_svr = grid_search_svr.score(X_test, Y_test)

# Ordenar los modelos de mejor a peor basado en R²
model_scores = [
    ('Linear Regression', r2_lr), 
    ('Gradient Boosting', r2_gb), 
    ('Ridge Regression', r2_ridge), 
    ('SVR', r2_svr)
]

sorted_models = sorted(model_scores, key=lambda x: x[1], reverse=True)

# Imprimir los modelos ordenados
for model_name, r2_score in sorted_models:
    print(f'Modelo: {model_name}, R²: {r2_score:.4f}')

Modelo: SVR, R²: 0.9297
Modelo: Linear Regression, R²: 0.9293
Modelo: Ridge Regression, R²: 0.9285
Modelo: Gradient Boosting, R²: 0.9252


Podemos observar que todos los modelos presentan valores de R² muy similares. Esto implica que no podemos basarnos únicamente en esta métrica para elegir el mejor modelo.

Además, la elección del modelo óptimo puede variar entre iteraciones debido a la influencia de diferentes factores o estrategias heurísticas utilizadas durante el proceso. Esto significa que un modelo que fue considerado el mejor en una ejecución podría no serlo en otra, como ya se ha comprobado en análisis previos.

## Persistencia del modelo

In [95]:
# Crear las carpetas si no existen
directorios = [
    'Persistencia',
    'Persistencia/Regresion',
    'Persistencia/Clasificacion',
    'Persistencia/clusterizacion',
]

for directorio in directorios:
    if not os.path.exists(directorio):
        os.makedirs(directorio)

In [96]:
# Guardar los mejores modelos
joblib.dump(best_lr_reg, 'Persistencia/Regresion/lr.pkl')
joblib.dump(best_gb_reg, 'Persistencia/Regresion/gb.pkl')
joblib.dump(best_ridge_grid, 'Persistencia/Regresion/ridge.pkl')
joblib.dump(best_svr, 'Persistencia/Regresion/svr.pkl')

# Guardar los conjuntos de datos
df.to_csv('Persistencia/Regresion/df.csv', index=False)